In [ ]:
import feedparser
import requests
from sqlalchemy import create_engine, Column, Integer, String, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [ ]:
Base = declarative_base()
# Create the SQLAlchemy engine and bind it to the Flask app
class Video(Base):
    __tablename__ = 'videos'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    url = Column(String)
    thumbnail = Column(String)
    tags = Column(String)
    embed = Column(String)
    def __repr__(self):
        return f'<Video {self.title}>'

In [ ]:
engine = create_engine('sqlite:///data.db')
Base.metadata.create_all(engine)


In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
url = 'http://www.pornhub.com/rss'
response = requests.get(url)

if response.status_code == 200:
    rss_feed = response.text

    # Parse the RSS feed
    feed = feedparser.parse(rss_feed)
    print(feed)
    # Filter entries containing the keyword 'mom'
    mom_entries = [
        entry for entry in feed.entries if 'mom' in entry.title.lower() or 'mom' in entry.tags ]

    # Print the number of filtered entries
    print(f"Number of entries containing 'mom': {len(mom_entries)}")

    # Print the titles of the filtered entries
    for entry in mom_entries:
        print(entry.title)
        print(entry.link)
        print(entry.thumb)
        print(entry.tags[0].term)
        print(entry.embed)
else:
    print(f'Failed to fetch RSS feed. Status code: {response.status_code}')


In [ ]:
for entry in mom_entries:
    Video_Entry = Video(
        title=entry.title,
        url=entry.link,
        thumbnail=entry.thumb,
        tags=entry.tags[0].term,
        embed=entry.embed
    )
    session.add(Video_Entry)

# Commit the changes to the database
session.commit()

In [ ]:
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

@app.route('/')
def index():
    videos = Video.query.all()
    return render_template('index.html', videos=videos)


In [ ]:
if __name__ == '__main__':
    app.run()